In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/brandonmanley/Desktop/PhD/oam_pheno/dijet_dsa')
import dsa_mc.dijet as dijet
import random
from IPython.display import display, Math
from tabulate import tabulate

# setup plotting
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 16
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["axes.labelsize"] = 16  
plt.rcParams["xtick.labelsize"] = 14  
plt.rcParams["ytick.labelsize"] = 14
plt.rcParams["axes.titlesize"] = 18  
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsmath}"
%config InlineBackend.figure_format = 'retina'

In [2]:
dj = dijet.DIJET(fit_type='ones', lambdaIR=0.3)


--> loaded unpol. amp. data from /dipoles/narr_ymin4.61_ymax14.91_AAMS09.dat
--> loaded pol. amp. data from /dipoles/d05-rc/
--> loaded params from /dipoles/replica_params_ones.csv
--> loaded random moment params from /dipoles/moment_params_ones.csv
--> loaded replica 1


In [3]:
# matching fourier-bessel 
kins = dijet.Kinematics(x=0.01, z=0.01, Q=10, pT=10)
print(dj.fourier_bessel(kins, [[1,1,1,1]], 'N'))
print(dj.fourier_bessel(kins, [[1,1,1,1]], 'Qu'))

-3.1843025584539455
318.8384682301659


In [4]:
# matching coefficients
kins = dijet.Kinematics(x=0.01, z=0.3, Q=10, pT=2, y=0.1, delta=0.5)
kins.s = (kins.Q**2)/(kins.x*kins.y)

coeffs = ['A_TT', 'B_TT', 'A_LT', 'B_LT', 'C_LT']
coeffs += ['A_TT_unpolar', 'B_TT_unpolar', 'A_TmT_unpolar', 'B_TmT_unpolar', 'C_TmT_unpolar', 'A_LL_unpolar', 'B_LL_unpolar']
coeff_values = {coeff: dj.get_coeff(coeff, kins) for coeff in coeffs}

# ben_coeffs = {
#     'A_TT': -8.494285245553912e-07,
#     'B_TT': -3.444049066922065e-08,
#     'A_LT': 2.356197548067663e-05,
#     'B_LT': 8.214268057773021e-05,
#     'C_LT': -9.16784882331537e-07,
#     'A_TT_unpolar': 1.4975559569144442e-06,
#     'B_TT_unpolar': -1.8843366541503538e-07,
#     'A_TmT_unpolar': -1.0844370722483909e-06,
#     'B_TmT_unpolar': 1.3645196461088766e-07,
#     'C_TmT_unpolar': -4.3377482889935633e-07,
#     'A_LL_unpolar': 1.615215249961842e-06,
#     'B_LL_unpolar':6.061657486205812e-07
# }
ben_coeffs = {
    'A_TT': -8.494285245553912e-07,
    'B_TT': -3.444049066922065e-08,
    'A_LT': 1.0242675452081359e-07,
    'B_LT': 1.595462973023054e-07,
    'C_LT': -9.167848823315369e-07,
    'A_TT_unpolar': 1.4975559569144442e-06,
    'B_TT_unpolar': -1.8843366541503538e-07,
    'A_TmT_unpolar': -1.0844370722483909e-06,
    'B_TmT_unpolar': 1.3645196461088766e-07,
    'C_TmT_unpolar': -4.3377482889935633e-07,
    'A_LL_unpolar': 1.615215249961842e-06,
    'B_LL_unpolar':6.061657486205812e-07
}

for coeff in coeffs:
    z = ((ben_coeffs[coeff] - coeff_values[coeff])/coeff_values[coeff])*100 
    print(coeff, z)

N_11 0.07218212952448869
N_00 0.0880932668706548
G2_11 18.128900397854043
Qs_11 22.84417434459833
Qs_00 27.852671486809697
Qu_11 22.84417434459833
Qu_00 27.852671486809697
Qd_11 22.84417434459833
Qd_00 27.852671486809697
A_TT -0.026270469113025767
B_TT -0.026270469103184403
A_LT -0.026270469110135003
B_LT -0.026270469107837025
C_LT -0.026270469114091234
A_TT_unpolar -0.02627046915190597
B_TT_unpolar -0.026270469157515783
A_TmT_unpolar -0.02627046915186424
B_TmT_unpolar -0.026270469157510436
C_TmT_unpolar -0.02627046915187644
A_LL_unpolar -0.02627046915520491
B_LL_unpolar -0.026270469152466197


In [5]:
# matching final results
kins = dijet.Kinematics(x=0.01, z=0.3, Q=10, pT=2, y=0.1, delta=0.5)
kins.s = (kins.Q**2)/(kins.x*kins.y)

weights = ['1', 'cos(phi_kp)', 'cos(phi_Dp)', 'cos(phi_Dp)cos(phi_kp)', 'sin(phi_Dp)sin(phi_kp)']
nums = [dj.get_angle_integrated_xsec(kins, weight=w, kind='num') for w in weights]
den = dj.get_angle_integrated_xsec(kins, weight='1', kind='den')
corrs = {w: nums[iw]/den for iw, w in enumerate(weights)}


ben_corrs = {
    '1': -0.018930764988144055,
    'cos(phi_kp)': 0.0008059486850585433,
    'cos(phi_Dp)': -9.594462866617396e-05,
    'cos(phi_Dp)cos(phi_kp)': 0.00015692448852636342,
    'sin(phi_Dp)sin(phi_kp)': -0.0009017194455850274
}


for w in weights:
    z = ((ben_corrs[w] - corrs[w])/corrs[w])*100 
    print(w, round(ben_corrs[w],8) , round(corrs[w], 8), round(z, 4))

N_11 0.07218212952448869
N_00 0.0880932668706548
G2_11 18.128900397854043
Qs_11 22.84417434459833
Qs_00 27.852671486809697
Qu_11 22.84417434459833
Qu_00 27.852671486809697
Qd_11 22.84417434459833
Qd_00 27.852671486809697
1 -0.01893076 -0.01893076 0.0
cos(phi_kp) 0.00080595 0.00080595 0.0
cos(phi_Dp) -9.594e-05 -9.594e-05 0.0
cos(phi_Dp)cos(phi_kp) 0.00015692 0.00015692 0.0
sin(phi_Dp)sin(phi_kp) -0.00090172 -0.00090172 0.0


In [6]:
coeff_eqs = ['13a','13b','15a','15b','15c','23a','23b', '23c','23d','23e','23f','23g']
corr_eqs = ['25a','25b','25c','25d','25e']

table_data = [[eq, bv, mv, bv/mv] for eq, bv, mv in zip(coeff_eqs, ben_coeffs.values(), coeff_values.values())]
table_data += [[eq, bv, mv, bv/mv] for eq, bv, mv in zip(corr_eqs, ben_corrs.values(), corrs.values())]

headers = ['eqn.', 'ben', 'me', 'ben/me']

# print(tabulate(table_data, headers=headers, tablefmt="grid"))
print(tabulate(table_data, headers=headers))




eqn.             ben            me    ben/me
------  ------------  ------------  --------
13a     -8.49429e-07  -8.49652e-07  0.999737
13b     -3.44405e-08  -3.44495e-08  0.999737
15a      1.02427e-07   1.02454e-07  0.999737
15b      1.59546e-07   1.59588e-07  0.999737
15c     -9.16785e-07  -9.17026e-07  0.999737
23a      1.49756e-06   1.49795e-06  0.999737
23b     -1.88434e-07  -1.88483e-07  0.999737
23c     -1.08444e-06  -1.08472e-06  0.999737
23d      1.36452e-07   1.36488e-07  0.999737
23e     -4.33775e-07  -4.33889e-07  0.999737
23f      1.61522e-06   1.61564e-06  0.999737
23g      6.06166e-07   6.06325e-07  0.999737
25a     -0.0189308    -0.0189308    1
25b      0.000805949   0.000805949  1
25c     -9.59446e-05  -9.59446e-05  1
25d      0.000156924   0.000156924  1
25e     -0.000901719  -0.000901719  1
